### Cache vs Persist 방식의 차이
- Spark에서 cache() 와 persist() 는 둘 다 RDD나 DataFrame을 메모리에 저장해 재사용하기 위한 메서드입니다.
- 하지만 저장 수준(Storage Level) 에서 차이가 있습니다.

#### 1. cache()

- 사실상 persist(StorageLevel.MEMORY_ONLY) 의 축약 버전입니다. 즉, 메모리에만 저장합니다.
- 메모리가 부족하면 캐시되지 않은 파티션은 필요할 때마다 다시 계산합니다.

- 사용 예:
  - rdd.cache()
 
##### 특징
- 빠름 (메모리 접근만 하기 때문)
- 하지만 메모리가 부족하면 디스크에 저장하지 않고 → 다시 연산 발생 → 재계산 비용 발생 가능

#### 2. persist()

- 사용자가 저장 수준(StorageLevel) 을 직접 지정할 수 있습니다.
- 예:
  - rdd.persist(StorageLevel.MEMORY_AND_DISK)

- 주요 Storage Level 옵션:
  - MEMORY_ONLY (기본, cache()와 동일)
  - MEMORY_AND_DISK (메모리에 못 올리면 디스크에 저장)
  - DISK_ONLY (디스크에만 저장)
  - _SER 옵션 (직렬화 저장 → 공간 절약, CPU 사용 증가)
  - _2 옵션 (복제본을 2개 저장)

#### cache() vs persist() 차이

| 구분         | cache()                          | persist()                           |
|--------------|----------------------------------|--------------------------------------|
| 기본 저장 방식 | 메모리(MEMORY_ONLY)              | 사용자가 선택 가능                   |
| 디스크 사용   | ❌ (메모리 부족 시 재계산)        | ⭕ (옵션 지정 시 가능)                |
| 직렬화 옵션   | ❌                               | ⭕ (e.g. MEMORY_ONLY_SER)             |
| 유연성       | 낮음                             | 높음                                 |

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 09:22:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/29 09:22:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/29 09:22:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/08/29 09:22:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
filepath = "/home/hadoop/python_code/spark/restaurant_reviews.csv"

In [3]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [4]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [5]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    
    return category, reviews

In [7]:
category_review_rdd = filtered_lines.map(parse)
category_review_rdd.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [8]:
result1 = category_review_rdd.reduceByKey(lambda x, y : x + y)
result1.collect()

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [9]:
result_sum_rdd = category_review_rdd.mapValues(lambda x : (x, 1))
result_sum_rdd.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [10]:
# persist 사용하기
category_reviews_rdd_persist = filtered_lines.map(parse).persist() # 메모리에 상주하는 RDD
category_reviews_rdd_persist.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [11]:
result1 = category_reviews_rdd_persist.reduceByKey(lambda x, y : x + y)
result1.collect()

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [12]:
result2 = category_reviews_rdd_persist.mapValues(lambda x : (x, 1))
result2.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [1]:
sc.stop()

NameError: name 'sc' is not defined